# Streaming Parquet Files to Delta Table
This notebook reads Parquet files as a stream and writes them to a Delta table with schema evolution enabled.

In [ ]:
# Configure spark to handle schema evolution
spark.conf.set("spark.sql.streaming.schemaInference", True)
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", True)

In [ ]:
# Read the streaming parquet files
stream_df = spark.readStream.format("parquet") \
    .option("mergeSchema", "true") \
    .load("abfss://external-location@databricksdevstgacc.dfs.core.windows.net/files2/")

In [ ]:
# Write the stream to delta table
query = stream_df.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option("checkpointLocation", "abfss://external-location@databricksdevstgacc.dfs.core.windows.net/checkpoints/stream_test") \
    .option("mergeSchema", "true") \
    .table("dev_catalog.default.stream_test")

In [ ]:
# Wait for the streaming query to terminate
query.awaitTermination()